[View in Colaboratory](https://colab.research.google.com/github/planewave/coherent_receiver_with_CNN/blob/master/CNN_for_symbol_timing.ipynb)

dependency is lost. will need `sig_gen5_new_timing.m` to generate `sto10dB.mat`


In [0]:
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu90' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
!rm -rf test_repo
!git clone https://github.com/planewave/test_repo.git

In [0]:
USE_CUDA = True
NUM_EPO = 50

dataRead = io.loadmat('test_repo/sto10dB.mat')

offset = dataRead['offset'] #(1024, 1) int64 
sig = (dataRead['x']) #(1024, 1, 16, 64)

offset = Variable(torch.from_numpy(offset+8).squeeze())
sig = Variable(torch.from_numpy(sig)) # floatTensor
offset.cuda().shape
# type(offset)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.sto_cnn = nn.Sequential(
            nn.Conv2d( 1, 16, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0)),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0)),
            nn.ReLU(),
            nn.Conv2d(16,  4, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d( 4,  1, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)),
            nn.ReLU(),
        )
    def forward(self, sig):
        out_cnn = self.sto_cnn(sig)
        out_mean = out_cnn.squeeze().mean(2)
        return nn.functional.softmax(out_mean, dim=1)
    
net = Net()
if USE_CUDA:
    offset = offset.cuda()
    sig = sig.cuda()
    net.cuda() # put it before optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)

# Training

In [0]:
for epoch in range(NUM_EPO):
    netout = net(sig)
    target = offset
    loss = loss_func(netout, target)
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step()
    if epoch%5 == 0:
        prediction = torch.max(F.softmax(netout, 1), 1)[1]
        accu = torch.sum(prediction.data == target.data)/prediction.shape[0]
        print('Epoch:', epoch, 'loss:', loss, 'accu', accu)


In [13]:
sig = (dataRead['x'])
sig = Variable(torch.from_numpy(sig))
m = nn.Sequential(
            nn.Conv2d( 1, 16, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0)),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0)),
            nn.ReLU(),
            nn.Conv2d(16,  4, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d( 4,  1, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0)),
            nn.ReLU(),
        )
output = m(sig)
output = output.squeeze().mean(2)
output.shape

# m2 = nn.Softmax(dim=1)
# output = m2(output)
# output[0,:]
# out_cnn = self.sto_cnn(sig)
# out_mean = out_cnn.squeeze().mean(2)
netout = nn.functional.softmax(output, dim=1)
prediction = torch.max(F.softmax(netout, 1), 1)[1]
offset = dataRead['offset']
target = Variable(torch.from_numpy(offset).squeeze())
accu = torch.sum(prediction.data == target.data)/prediction.shape[0]
accu

# print(target)
loss_func = nn.CrossEntropyLoss()

loss = loss_func(netout, target)
loss

Variable containing:
 2.7727
[torch.FloatTensor of size 1]